In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Model and API Key

In [2]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Agents & Tools

In [3]:
# Additional imports for agent with built-in tools.
from langchain.agents import initialize_agent, load_tools

In [4]:
# Initialize the model.
llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.3)

# Set up the built-in wikipedia tool.
tools = load_tools(['wikipedia'], llm=llm)

# Initialize the agent.
agent = initialize_agent(tools, agent="chat-zero-shot-react-description", verbose=True, llm=llm)

# Define a query as a string inside a dictionary.
query = {"input": "What is the age of the mayor of the capital of Canada?"}

result = agent.run(query)
print(result)

/Users/slemons/anaconda3/envs/dev/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
/Users/slemons/anaconda3/envs/dev/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I can use Wikipedia to find information about the mayor of the capital of Canada.

Action:
```
{
  "action": "Wikipedia",
  "action_input": "Mayor of Ottawa"
}
```
Observation: Page: Mayor of Ottawa
Summary: The mayor of Ottawa is head of the executive branch of the Ottawa City Council. The mayor is elected alongside the city council every four years on the fourth Monday of October; there are no term limits. While in office, mayors are styled His/Her Worship. 
Mark Sutcliffe has served as the 59th and current mayor of Ottawa since taking office on November 15, 2022, following the 2022 municipal election.

Page: Ottawa
Summary: Ottawa ( , ; Canadian French: [ɔtawɑ]) is the capital city of Canada. It is located in the southern portion  of the province of Ontario, at the confluence of the Ottawa River and the Rideau River. Ottawa borders Gatineau, Quebec, and forms the core of the Ottawa–Gatineau census metropolitan area (CMA) and the Natio

/Users/slemons/anaconda3/envs/dev/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/slemons/anaconda3/envs/dev/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Mark Sutcliffe
Summary: Mark Sutcliffe (born July 14, 1968) is a Canadian politician who has been the 59th mayor of Ottawa since 2022. Before entering politics, he hosted Ottawa Today on 1310News radio. Sutcliffe is the first Chinese Canadian and person from a visible minority group to serve as mayor of Ottawa.

Page: Peter Sutcliffe
Summary: Peter William Sutcliffe (2 June 1946 – 13 November 2020), also known as Peter Coonan, was an English serial killer who was convicted of murdering thirteen women and attempting to murder seven others between 1975 and 1980.: 144  He was dubbed in press reports as the Yorkshire Ripper, an allusion to the Victorian serial killer Jack the Ripper. He was sentenced to twenty concurrent sentences of life imprisonment, which were converted to a whole life order in 2010. Two of Sutcliffe's murders took place in Manchester; all the others were in West Yorkshire. Criminal psychologist Dr. David Holmes characterised Sutcliffe as being an "e

# Agent with Multiple Tools

In [5]:
# Additional imports for agent with built-in tools.
from langchain.agents import initialize_agent, load_tools

In [7]:
# Initialize the model.
llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.3)

# Set up the built-in wikipedia and open weather tools.
tools = load_tools(['wikipedia', "openweathermap-api"], llm=llm)

# Initialize the agent.
agent = initialize_agent(tools,
                         verbose=True,
                         handle_parsing_errors = True,
                         max_iterations=10,
                         llm=llm)

# Define a query as a string inside a dictionary.
query = {"input": "What is the current weather in the city where penicillin was first isolated?"}

# Run the agent using the query.
result = agent.run(query)
print(result)



> Entering new AgentExecutor chain...
I need to find out the city where penicillin was first isolated before I can fetch the current weather information.
Action: Wikipedia
Action Input: "penicillin"
Observation: Page: Penicillin
Summary: Penicillins (P, PCN or PEN) are a group of β-lactam antibiotics originally obtained from Penicillium moulds, principally P. chrysogenum and P. rubens. Most penicillins in clinical use are synthesised by P. chrysogenum using deep tank fermentation and then purified. A number of natural penicillins have been discovered, but only two purified compounds are in clinical use: penicillin G (intramuscular or intravenous use) and penicillin V (given by mouth). Penicillins were among the first medications to be effective against many bacterial infections caused by staphylococci and streptococci. They are still widely used today for different bacterial infections, though many types of bacteria have developed resistance following extensive use.
Ten percent of th

# Creating Custom Tools

In [8]:
# Additional imports for creating tools.
from langchain.tools import tool
from langchain.agents import initialize_agent
from langchain.chains import APIChain, LLMMathChain

In [9]:
# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)

# Text description of API spec.
spec = """URL STRUCTURE
        Just hit http://numbersapi.com/number/type to get a plain text response, where
        type is one of trivia, math, date, or year. Defaults to trivia if omitted.
        number is an integer, or the keyword random, for which we will try to return
        a random available fact, or a day of year in the form month/day 
        (eg. 2/29, 1/09, 04/1), if type is date ranges of numbers"""

# Create the API chain from the spec, using the LLM.
api_chain = APIChain.from_llm_and_api_docs(
    llm,
    api_docs=spec,
    verbose=True,
    limit_to_domains=["https://numbersapi.com", "http://numbersapi.com"],
)

# Define a tool for querying the Numbers API.
@tool("numbers_API")
def numbers_tool(query: str) -> str:
    """Look up fun facts about numbers or dates."""
    result = api_chain.run(query)
    return result

# Initialize the math chain using the LLM.
math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

# Define a tool for doing math.
@tool("math_tool")
def math_tool(query: str) -> str:
    """Perform math calculations."""
    result = math_chain.run(query)
    return result

# Store the tools in a list.
tools = [math_tool, numbers_tool]

# Initialize the agent.
agent = initialize_agent(tools, agent="zero-shot-react-description", verbose=True, llm=llm)

# Define a query as a string inside a dictionary.
query = {"input": 'What is a fun fact about the sum of 22 and 35?'}

result = agent.run(query)
print(result)



> Entering new AgentExecutor chain...
I can use the math_tool to calculate the sum of 22 and 35, and then use the numbers_API to find a fun fact about that number.
Action: math_tool
Action Input: "22 + 35"

> Entering new LLMMathChain chain...
22 + 35```text
22 + 35
```
...numexpr.evaluate("22 + 35")...

Answer: 57
> Finished chain.

Observation: Answer: 57
Thought:Now I can use the numbers_API to find a fun fact about the number 57.
Action: numbers_API
Action Input: "57"

> Entering new APIChain chain...
http://numbersapi.com/57
57 is the number of cm that the smallest man measured, Gul Mohammed (1957-1997) of India.

> Finished chain.

Observation: The number 57 is the measurement in centimeters of the smallest man, Gul Mohammed, from India.
Thought:I now know the final answer
Final Answer: The sum of 22 and 35 is 57, and a fun fact about that number is that it is the measurement in centimeters of the smallest man, Gul Mohammed, from India.

> Finished chain.
The sum of 22 and 35 i